In [1]:
import oauth
email_address = "indecorpredicate@gmail.com"

In [2]:
Indecor = oauth.OAuthGmailUser("ben@terminallabs.com", cred_path=oauth.CRED_PATH, token_path=oauth.TOKEN_PATH, scopes=["modify"])

starting a new service


In [3]:
Indecor.get_messages(max_results=4)

[('17b83a5342d9a2f4', 'We’re sorry we missed\u202fyou'),
 ('17b832f8da36773f', 'REMINDER: Adobe wants your input | $100 gift code if you qualify!'),
 ('17b7e5bbda5daa5e', 'Announcing HashiConf Global speakers: Stripe, Target, Workday, Petco & more'),
 ('17b7e43818c82a66', 'Keeping IT & Cloud Governance, Security and Costs in check')]